In [1]:
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import requests
import csv
import urllib.request
import urllib.parse
import bs4
import googlemaps
import pandas as pd
from selenium import webdriver

driver_loc = "../externlib/chromedriver/chromedriver.exe"

options = webdriver.ChromeOptions()

options.add_argument("window-size=1920x1080")

driver = webdriver.Chrome(driver_loc, options=options)

baseUrl = "https://www.diningcode.com/"

driver.get(baseUrl)

driver.current_url

time.sleep(10)
# 팝업창 종료
closePath = '//*[@id="div_popup"]/div/div[3]'
driver.find_element_by_xpath(closePath).click()

# 검색 하기
searchPath = '//*[@id="txt_keyword"]'
driver.find_element_by_xpath(searchPath).send_keys("강남구 신사동")

searchPath = '//*[@id="t_search"]/button'
driver.find_element_by_xpath(searchPath).click()

# 결과 더보기 클릭
time.sleep(2)
arrowBtn = '//*[@id="root"]/div/div/div[2]/div[3]/div'
driver.find_element_by_xpath(arrowBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

fileName = '../dataset/diningcode.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")



C:\Users\SMART18\AppData\Local\Temp/ipykernel_27832/1896495771.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_loc, options=options)
C:\Users\SMART18\AppData\Local\Temp/ipykernel_27832/1896495771.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(closePath).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_27832/1896495771.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(searchPath).send_keys("강남구 신사동")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_27832/1896495771.py:40: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(searchPath).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_27832/1896495771.p